In [1]:
from util import *
model_dir = 'saved_model/2xbidirectionalLSTM'
_SAMPLING_RATE = 44100
sample_length = 50
vocab_size = 128

/home/mew/anaconda3/envs/tf/lib/python3.9/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [2]:
data_dir = pathlib.Path("./data/beeth/")
filenames = glob.glob(str(data_dir/'*.mid*'))
print('Number of files:', len(filenames))

Number of files: 29


In [3]:
model = tf.keras.models.load_model(model_dir)
model.summary()

2022-09-09 21:52:37.760106: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-09 21:52:38.081416: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-gnu/gazebo-11/plugins:/opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/ros/foxy/opt/rviz_ogre_vendor/lib:/opt/ros/foxy/lib/x86_64-linux-gnu:/opt/ros/foxy/lib
2022-09-09 21:52:38.081438: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Ski

Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 50, 3)]      0           []                               
                                                                                                  
 bidirectional_40 (Bidirectiona  (None, 50, 128)     34816       ['input_21[0][0]']               
 l)                                                                                               
                                                                                                  
 dropout_3 (Dropout)            (None, 50, 128)      0           ['bidirectional_40[0][0]']       
                                                                                                  
 bidirectional_41 (Bidirectiona  (None, 128)         98816       ['dropout_3[0][0]']       

In [4]:
def predict_next_note(
        notes: np.ndarray,
        keras_model: tf.keras.Model,
        temperature: float = 1.0) -> int:
    """Generates a note IDs using a trained sequence model."""

    assert temperature > 0

    # Add batch dimension
    inputs = tf.expand_dims(notes, 0)

    predictions = model.predict(inputs)
    # print(predictions)
    pitch_logits = predictions[0]
    step = predictions[1]
    duration = predictions[2]

    pitch_logits /= temperature
    pitch = tf.random.categorical(pitch_logits, num_samples=1)
    pitch = tf.squeeze(pitch, axis=-1)
    duration = tf.squeeze(duration, axis=-1)
    step = tf.squeeze(step, axis=-1)

    # `step` and `duration` values should be non-negative
    step = tf.maximum(0, step)
    duration = tf.maximum(0, duration)

    return int(pitch), float(step), float(duration)

def generate_new_notes(filepath, temperature=3.0, num_predictions=120):
    raw_notes = midi_to_notes(filepath)
    key_order = ['pitch', 'step', 'duration']
    sample_notes = np.stack([raw_notes[key] for key in key_order], axis=1)
    # The initial sequence of notes; pitch is normalized similar to training
    # sequences
    input_notes = (
            sample_notes[:sample_length] / np.array([vocab_size, 1, 1]))
    generated_notes = []
    prev_start = 0
    for _ in range(num_predictions):
        [pitch, step, duration] = predict_next_note(input_notes, model, temperature)
        start = prev_start + step
        end = start + duration
        input_note = (pitch, step, duration)
        generated_notes.append((*input_note, start, end))
        input_notes = np.delete(input_notes, 0, axis=0)
        input_notes = np.append(input_notes, np.expand_dims(input_note, 0), axis=0)
        prev_start = start

    generated_notes = pd.DataFrame(generated_notes, columns=(*key_order, 'start', 'end'))
    return generated_notes

## select input and genetrate

In [26]:
inputnote=filenames[12]#select input file here
generated_notes = generate_new_notes(inputnote,temperature=3.0, num_predictions=50)


1/1 [==============================] - 0s 25ms/step


## INPUT NOTES

In [27]:
notes = midi_to_notes(inputnote)
input_notes_file = 'input.midi'
input_notes_pm = notes_to_midi(
    notes, out_file=input_notes_file, instrument_name='Acoustic Grand Piano')
play_audio(input_notes_pm,int(notes.iloc[sample_length].end))

fluidsynth: warning: SDL2 not initialized, SDL2 audio driver won't be usable
fluidsynth: error: Unknown integer parameter 'synth.sample-rate'


## Generated notes

In [28]:
out_file = 'output.mid'
out_pm = notes_to_midi(
    generated_notes, out_file=out_file, instrument_name='Acoustic Grand Piano')
play_audio(out_pm,int(generated_notes.iloc[-1,-1]))

fluidsynth: warning: SDL2 not initialized, SDL2 audio driver won't be usable
fluidsynth: error: Unknown integer parameter 'synth.sample-rate'
